# Deep Learning

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
!control sysdm.cpl

In [ ]:
!pip uninstall tensorflow_gpu

In [1]:
import tensorflow as tf
hello = tf.constant('Hello, TensorFlow!')
sess = tf.Session()
print(sess.run(hello))

ImportError: Traceback (most recent call last):
  File "C:\Users\Damien\Anaconda3\lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "C:\Users\Damien\Anaconda3\lib\site-packages\tensorflow\python\pywrap_tensorflow_internal.py", line 18, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "C:\Users\Damien\Anaconda3\lib\site-packages\tensorflow\python\pywrap_tensorflow_internal.py", line 17, in swig_import_helper
    return importlib.import_module(mname)
  File "C:\Users\Damien\Anaconda3\lib\importlib\__init__.py", line 126, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
ImportError: DLL load failed: Une routine d’initialisation d’une bibliothèque de liens dynamiques (DLL) a échoué.


Failed to load the native TensorFlow runtime.

See https://www.tensorflow.org/install/install_sources#common_installation_problems

for some common reaso

In [ ]:
from sklearn import preprocessing

from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping

In [ ]:
#Loading dataset
train = pd.read_csv('train.csv')
#Scaling improves the optimization of the neural network
predictors = preprocessing.scale(train.drop(columns=['label']).as_matrix())
#Get target variable as dummy variable
target = to_categorical(train['label'])

print("Shapes of predictors and target arrays:", predictors.shape, target.shape)

In [ ]:
%%time

#Creating the neural network model
n_cols = predictors.shape[1]

model = Sequential()
model.add(Dense(1000, activation='relu', input_shape=(n_cols,)))
model.add(Dense(1000, activation='relu'))
model.add(Dense(10, activation='softmax'))  #10 digits to recognize

#Model compiling and fitting
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

#Performance monitor
early_stopping_monitor = EarlyStopping(patience=3)

#Fitting on data
model.fit(predictors, target, validation_split=0.3, epochs=20, callbacks=[early_stopping_monitor] )

In [56]:
#Prepare the submission dataset
test = pd.read_csv('../input/test.csv')
submission = preprocessing.scale(test.as_matrix())
print("Submission dataset shape: ", submission.shape)

#Make predictions on the test set
predictions = model.predict(submission)  #Contains probabilities for each 10 digits
print("Predictions shape (contains probabilities for each digit type) :", predictions.shape)

#Creating the submission dataframe
submission_df = pd.DataFrame(predictions)
submission_df['ImageId'] = pd.Series(range(1, 28001))
#Take the corresponding column label for the max value (probability) of the row
submission_df['label'] = submission_df.drop(columns=['ImageId']).idxmax(1)
print("10 first predictions on the submission digits: \n", submission_df[['label']].head(10))

#Print the 10 first images of the submission dataset
images_subset = test.iloc[:10, :]
for i, row in images_subset.iterrows():
    plt.subplot(2, 5, i+1)
    pixels = row.values.reshape((28,28))
    plt.imshow(pixels, cmap='gray')
    plt.xticks([])
    plt.yticks([])
plt.title('10 first images from the submission dataset')
plt.show()

/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


Submission dataset shape:  (28000, 784)
Predictions shape (contains probabilities for each digit type) : (28000, 10)


In [60]:
#Save the submission as csv
submission_df.loc[:, ['ImageId', 'label']].to_csv('submission_deep_learning_DB.csv', index=False)